In [5]:
from scipy.stats import norm
from pandas import DataFrame
import numpy as np
import pandas as pd

def bsCall(S, K, r, s, T):
    d1 = (np.log(S/K) + (r + 0.5*s**2)*T)/(s*np.sqrt(T))
    d2 = d1 - s*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    
def bsPut(S, K, r, s, T):
    d1 = (np.log(S/K) + (r + 0.5*s**2)*T)/(s*np.sqrt(T))
    d2 = d1 - s*np.sqrt(T)
    return -S*norm.cdf(-d1) + K*np.exp(-r*T)*norm.cdf(-d2)

vol = pd.read_csv('IV.csv', parse_dates=['Date']).iloc[1:, :].set_index('Date')
spx = pd.read_csv('data.csv', parse_dates=['Date'])[['Date', 'SPX']].set_index('Date')
param = pd.read_csv('LIBOR.csv', header=5, parse_dates=[0]).iloc[:, :2].set_index('Date').rename(columns={'PX_ASK': '1ML'}).merge(spx, on='Date')

dates = param.index.intersection(vol.index)
param = param.loc[dates]
param = param.assign(FWD=np.exp(param['1ML']/1200)*param['SPX'])
vol = vol.loc[dates]
strike = DataFrame(np.array([param['FWD'] + Delta for Delta in [-90, -75, -60, -50, -40, -25, -10, 0, 10, 25, 40, 50, 60, 75, 90]]).T, 
                  columns=vol.columns, index=vol.index)

In [ ]:
%%time

m, n = vol.shape
pv = np.empty_like(vol)
for i in range(m):
    for j in range(n):
        pricingFnc = bsCall if vol.columns[j][0] == 'C' else bsPut
        if i==235:
            S = param['SPX'].iloc[i], 
            K = strike.iloc[i, j], 
            r = param['1ML'].iloc[i]/100, 
            s = vol.iloc[i, j]/100, 
            T = 1/12
            
            d1 = (np.log(S/K) + (r + 0.5*s**2)*T)/(s*np.sqrt(T))
            d2 = d1 - s*np.sqrt(T)
            put = -S*norm.cdf(-d1) + K*np.exp(-r*T)*norm.cdf(-d2)
            print(d1, d2, put)
            break
        pv[i, j] = pricingFnc(  S=param['SPX'].iloc[i], 
                                K=strike.iloc[i, j], 
                                r=param['1ML'].iloc[i]/100, 
                                s=vol.iloc[i, j]/100, 
                                T=1/12)
        
pv = DataFrame(pv, columns=vol.columns, index=vol.index)
pv.to_csv('pv.csv')

In [39]:
from sklearn.preprocessing import normalize

pv = pd.read_csv('pv.csv', index_col=0)
pv.index = strike.index

import seaborn as sns

np.where(pv.isna())

(array([234, 234, 234, 234, 234, 234, 234, 234, 234, 234, 234, 234, 234,
        234, 235, 235, 235, 235, 235, 235, 235, 235, 235, 235, 235, 235,
        235, 235, 236, 236, 236, 236, 236, 236, 236, 236, 236, 236, 236,
        236, 236, 236, 237, 237, 237, 237, 237, 237, 237, 237, 237, 237,
        237, 237, 237, 237]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  1,  2,  3,
         4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  1,  2,  3,  4,  5,  6,
         7,  8,  9, 10, 11, 12, 13, 14,  1,  2,  3,  4,  5,  6,  7,  8,  9,
        10, 11, 12, 13, 14]))